# ```torch.autograd```에 대해 알아보자  
- URL from : https://tutorials.pytorch.kr/beginner/blitz/autograd_tutorial.html
- URL from : https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html

## 배경 지식(background)  
신경망(NN : Neural Network)은 어떤 입력 데이터에 대해 실행되는 중첩(nested)된 함수들의 모음(collection)이다. 이 함수들은 PyTorch에서 tensor로 저장되는, 가중치(weight)와 편향(bias)으로 구성된 매개변수들로 정의된다.  

신경망을 학습하는 건 2단계로 이루어진다.

순전파(forward propagaton)
- 순전파 단계에서, 신경망은 정답을 맞추기 위해 최선의 추측(best guess)을 한다. 이렇게 추측하기 위해 입력 데이터를 각 함수들에서 실행한다.

역전파(backward propagation)
- 역전파 단계에서, 신경망은 추측한 값에서 발생한 오류(error)에 비례하여(propagation) 매개변수들을 적절히 조절(adjust)한다. 출력(output)으로부터 역방향으로 이동하면서 오류에 대한 함수들의 매개변수들의 미분값(=변화도=gradient)을 수집하고, 경사하강법(gradient descent)을 사용하여 매개변수들을 최적화한다.

## PyTorch에서의 사용법
본 튜토리얼에서는 ```torchvision```에서 미리 학습된 resnet18 모델을 갖다 쓴다. 3채널짜리 이미지 채널과 64 x 64짜리 넓이를 갖는 이미지 하나를 표현하는 무작위의 데이터 tensor를 생성하고, 이에 상응하는 ```label(정답)```을 무작위 값으로 초기화한다. 미리 학습된 모델의 정답(label)은 (1, 1000)의 모양(shape)을 갖는다.  
+본 튜토리얼은 tensor를 CUDA로 이동하더라도 GPU에서는 동작하지 않고 CPU에서만 동작한다.

In [1]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

다음으로, 입력(input) 데이터를 모델의 각 층(layer)에 통과시켜 예측값(prediction)을 생성해보자. 이게 순전파 단계다.

In [16]:
prediction = model(data) # 순전파 단계(forward pass)

모델의 예측값과 그에 해당하는 정답(label)을 사용하여 오차(error=손실=loss)를 계산한다. 다음 단계는 신경망을 통해 이 에러를 역전파하는 과정이다. 오차(error) tensor에 ```.backward()```를 호출하면 역전파가 시작된다. 그 다음 Autograd가 매개변수(parameter)의 ```.grad```속성(attribute)에, 모델의 각 매개변수에 대한 변화도(gradient)를 계산하고 저장한다.

In [17]:
loss = (prediction - labels).sum()
loss.backward() # 역전파 단계(backward pass)

다음으로, 옵티마이저(optimizer)를 불러온다. 이 예제에서는 학습률(learning rate) 0.1과 모멘텀(momentum) 0.9를 갖는 SGD를 사용한다. optimizer에 모델의 모든 매개변수를 등록해보자.

In [18]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

마지막으로 ```.step```을 호출해서 경사 하강법(gradient descent)을 시작한다. optimizer는 ```.grad```에 저장된 변화도에 다라 각 매개변수를 조정(adjust)한다.

In [19]:
optim.step() # 경사 하강법(gradient descent)

## Autograd에서 미분(differentiation)  
```autograd```가 어떻게 변화도(gradient)를 수집하는지 살펴보자. ```requires_grad=True```를 갖는 2개의 tensor ```a```와 ```b```를 만들자. ```requires_grad=True```는 ```autograd```에 모든 연산(operation)들을 추적해야 한다고 알려준다.

In [20]:
import torch
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

이제 ```a```와 ```b```로부터 새로운 텐서 ```Q```를 만들자.  

In [21]:
Q = 3*a**3 - b**2

이제 ```a```, ```b```가 모두 신경망(NN)의 매개변수이고, ```Q```가 오차(error)라고 가정해보자. 신경망을 학습할 때, 매개변수들에 대한 오차의 변화도(gradient)를 구해야 한다.  

```Q```는 벡터(vector)이므로, ```Q.backward()```에 ```gradient```인자(argument)를 명시적으로 전달해야 한다. ```gradient```는 ```Q```와 같은 모양(shape)의 tensor로, ```Q``` 자기 자신에 대한 변화도(gradient)를 나타낸다.  

동일하게, ```Q.sum().backward()```와 같이 ```Q```를 스칼라(scalar) 값으로 집계(aggregate)한 뒤, 암시적으로 ```.backward()```를 호출할 수도 있다.

In [22]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

이제 변화도는 ```a.grad```와 ```b.grad```에 저장된다.

In [23]:
# 수집된 변화도가 올바른지 확인해보자
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])
